In [1]:
########################
# Imports
########################
from __future__ import division
import math
import random
import csv
import numpy as np
import nltk
import json
from sklearn import tree
from sklearn.tree import export_graphviz

print "Done"

Done


In [2]:
############################
# Retrieving data from file
###########################
Data = []
with open('forestfires.csv') as f:
    reader = csv.DictReader(f, delimiter=',')
    for row in reader:
        Data.append({"X": row["X"],"Y": row["Y"],"month": row["month"],
                     "day": row["day"],"FFMC": row["FFMC"] ,"DMC": row["DMC"]
                    ,"DC": row["DC"] ,"ISI": row["ISI"] ,"temp": row["temp"]
                    ,"RH": row["RH"] ,"wind": row["wind"] ,"rain": row["rain"]
                    ,"area": row["area"]})
print "Done"

Done


In [3]:
########################
# Create train/test-set
########################
large_set = ([])
small_set = ([])
large_numb = 0
small_numb = 0
count = 0
for fire in Data:
    count +=1
    if(float(fire["area"])>0):
        large_set.append([fire,'High'])
        large_numb +=1
    else:
        small_set.append([fire,'Low'])
        small_numb +=1
temp = (large_set+small_set)
print "Gone through "+ str(count) +" fires"
print "Number of Large Fires: "+ str(large_numb)
print "Number of Small fires: "+ str(small_numb)

Gone through 517 fires
Number of Large Fires: 270
Number of Small fires: 247


In [4]:
############################
# Features for Naive Bayes
############################

def NB_Features(data):
    features = {}
#     features["X"] =int(data["X"])
#     features["Y"] =int(data["Y"])
    features["Month"] =data["month"]
#     features["FFMC"] = int(np.round(float(data["FFMC"])))
#     features["DMC"] = int(np.round(float(data["DMC"])))
#     features["DC"] = int(np.round(float(data["DC"])))   
#     features["ISI"] = int(np.round(float(data["ISI"])))
#     features["Temperature"] =int(np.round(float(data["temp"])))
    features["Relative Humidity"] = (int)(data["RH"])
    features["Wind"] =int(np.round((float)(data["wind"])))
    features["Rain"] =data["rain"]
    return features

print "Done"

Done


In [6]:
##################################
# Training Naive Bayes Classifier
##################################
random.shuffle(temp)
test_buffer = (temp[:int(len(temp)/5)])
train_buffer =(temp[int(len(temp)/5):])

print "Length of data used: " + str(len(test_buffer)+len(train_buffer))
train_set = [(NB_Features(data), risk) 
             for (data,risk) in train_buffer]
test_set = [(NB_Features(data), risk) 
             for (data,risk) in test_buffer]
NBClassifier = nltk.NaiveBayesClassifier.train(train_set)
print "Classifier is trained"
print "Classifier accuracy: "+str(nltk.classify.accuracy(NBClassifier, test_set))
NBClassifier.show_most_informative_features(15)

Length of data used: 517
Classifier is trained
Classifier accuracy: 0.533980582524
Most Informative Features
       Relative Humidity = 53               High : Low    =      6.6 : 1.0
       Relative Humidity = 28               High : Low    =      5.9 : 1.0
       Relative Humidity = 48                Low : High   =      4.6 : 1.0
       Relative Humidity = 51                Low : High   =      3.6 : 1.0
       Relative Humidity = 24                Low : High   =      3.2 : 1.0
       Relative Humidity = 78                Low : High   =      3.2 : 1.0
       Relative Humidity = 32                Low : High   =      3.2 : 1.0
       Relative Humidity = 41               High : Low    =      2.8 : 1.0
       Relative Humidity = 26               High : Low    =      2.8 : 1.0
                   Month = 'mar'             Low : High   =      2.6 : 1.0
       Relative Humidity = 19                Low : High   =      2.5 : 1.0
       Relative Humidity = 46               High : Low    =      2

In [7]:
#############################
# Features for Decision tree
#############################

def Tree_Features(data):
    features = []
#     features.append(int(data["X"]))
#     features.append(int(data["Y"]))
#     features.append(data["month"]) # Virker ikke lige pt
#     features.append(float(data["FFCM"]))
#     features.append(float(data["DCM"]))
#     features.append(float(data["CM"]))
#     features.append(float(data["ISI"]))
#     features.append(float(data["temp"]))
    features.append(float(data["RH"]))
    features.append(float(data["wind"]))
    features.append(float(data["rain"]))
    return features

print "Done"

Done


In [8]:
#################
# Decision tree
#################
random.shuffle(temp)

# convert categories and PdDistricts to ints
firesX = [Tree_Features(fire[0]) for fire in temp]

# setup the classifiers
DTClassifier = tree.DecisionTreeClassifier()

# split the data into 90% traning and 10% test prediction
firesX_train = np.array(firesX[:int(len(firesX)*0.9)])
firesX_test = np.array(firesX[-int(len(firesX)*0.1):])
value = [fire[1] for fire in temp]
firesY_train = np.array(value[:int(len(value)*0.9)])
firesY_test = np.array(value[-int(len(value)*0.1):])

# # initialize of new arrays
# firesX_train_rearranged = np.zeros((len(firesX_train), 1))
# firesX_test_rearranged = np.zeros((len(firesX_test), 1))
# # # Rearrange the training and test data to two dimentional arrays [[features],[features]]
# count =0
# print firesX_train
# for x in firesX_train:
#     print x
#     firesX_train_rearranged[count] = x
#     count +=1
    
# count =0
# for x in firesX_test:
#     print x
#     firesX_test_rearranged[count] = x
#     count +=1

# # Train the classifiers and get perdictions from the test data
DTClassifier.fit(firesX_train, firesY_train)
Z = DTClassifier.predict(firesX_test)

numbOfCorrect =0
count=0
for ans in Z:
    if ans ==firesY_test[count]:
        numbOfCorrect+=1
    count+=1
print "{:0.2f}".format(numbOfCorrect/len(firesY_test)*100)

58.82


In [9]:
################################
# Output file for Decision tree
################################
from sklearn.externals.six import StringIO
with open("fire.dot", 'w') as f:
    f = tree.export_graphviz(DTClassifier, out_file=f)